# Classificatore Linear Support Vector

In [ ]:
# Numero di fold
num_folds = 10

# Inizializzazione delle liste per i valori di accuracy, F1 e coefficienti
accuracy_list = []
fold_f1_scores=[]
coefs_all=[]

# Ciclo sui fold
for i in range(num_folds):
    
    # Lettura dei dati di addestramento di test e training
    train_data = pd.read_csv(f"kfold_Combo_and_Difference/fold_{i}/train.tsv", sep='\t')
    test_data = pd.read_csv(f"kfold_Combo_and_Difference/fold_{i}/test.tsv", sep='\t')
    
    # Estrazione delle feature e delle labels per test e training
    X_train = train_data.drop("Order_Label", axis=1)
    y_train = train_data["Order_Label"]
    X_test = test_data.drop("Order_Label", axis=1)
    y_test = test_data["Order_Label"]
    
    # Estrazione nomi simbolici delle feature
    features_names = X_train.columns.tolist()
    
    # Scaling valori con MinMaxScaler
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    X_train_fold, y_train_fold = X_train_scaled, y_train
    X_test_fold, y_test_fold = X_test_scaled, y_test
    
    # Creazione del classificatore lineare Support Vector
    clf = LinearSVC(loss='squared_hinge', max_iter=3000)

    # Addestramento del classificatore su dati di training e test del fold corrente
    clf.fit(X_train_fold, y_train_fold)

    # Testing del classificatore sul test set per il fold corrente
    fold_accuracy = clf.score(X_test_fold, y_test_fold)
    
    # Inserisco l'accuracy corrente nella lista delle accuracy 
    accuracy_list.append(fold_accuracy)
    
    # Estraggo predizioni del modello
    y_pred = clf.predict(X_test_fold)
    
    # Estraggo ed inserisco i valori dell'f1 nella lista apposita
    fold_f1_score = f1_score(y_test_fold, y_pred, average='weighted')
    fold_f1_scores.append(fold_f1_score)
    
    print("Iterazione",i)
    
    print('Classification report:')
    # Stampa classification report
    print(classification_report(y_test_fold, y_pred))
    # Calcolo della media pesata delle F1 score per ogni classe
    test_f1_score = f1_score(y_test_fold, y_pred, average='weighted')
    
    print('Confusion matrix:')
    cm = confusion_matrix(y_test_fold, y_pred)
    # Mostra Confusion Matrix del fold
    cm_display = ConfusionMatrixDisplay(cm)
    cm_display.display_labels = clf.classes_
    cm_display.plot(cmap='Blues', values_format='.4g')
    plt.title(f"Confusion Matrix for Fold {i+1}")
    plt.show()
    
    # Feature importance

    # Estrago coefficienti del modello
    coefs = clf.coef_ 
    idx = 0
    class_coefs = coefs[idx]
    
    # Crea un dizionario che associa ciascun nome di feature al suo coefficiente corrispondente
    feature_importances = {feature_name: coef for feature_name, coef in zip(features_names, class_coefs)}
    
    # Aggiunge il dizionario alla lista che contiene i coefficienti per diverse classi o fold
    coefs_all.append(feature_importances)
    # Ordina le feature in base al loro valore
    sorted_feature_importances = dict(sorted(feature_importances.items(), key=lambda item: item[1], reverse=True))
    
    # Crea grafico con le 15 feature più discriminanti
    num_to_plot = 15
    print(f'Feature più utili per discriminare le classi')
    plt.barh(range(num_to_plot), list(sorted_feature_importances.values())[:num_to_plot], align='center')
    plt.yticks(range(num_to_plot), list(sorted_feature_importances.keys())[:num_to_plot])
    plt.gca().invert_yaxis()
    plt.show()
    
# Calcolo dell'accuracy e f1 media
mean_accuracy = np.mean(accuracy_list)
mean_f1_score=np.mean(fold_f1_scores)

print("Risultati medi su tutti i fold")
# Stampa dell'accuracy e F1 media
print(f"Mean Accuracy: {mean_accuracy}")
print(f"Mean F1: {mean_f1_score}")  

# Inizio processo di feature importance

list_of_dicts = coefs_al
# Conterrà liste di valori di ogni fold per ogni feature
result_dict = defaultdict(list)
# Per ogni dizionario nella lista
for d in list_of_dicts:
    # Per ogni chiave nel dizionario
    for key, value in d.items():
        # Aggiunge il valore alla lista corrispondente alla chiave
        result_dict[key].append(value)
        
# Per ogni chiave di result_dict, ovvero per ogni feature
for key, value in result_dict.items():
    # Calcola la media dei valori
    avg_value = sum(value) / len(value)
    # Aggiunge la media come valore della rispettiva feature (key) 
    result_dict[key] = avg_value
       
result_dict = dict(result_dict)
# Ordina le feature in base al loro valore
sorted_feature_importances = dict(sorted(result_dict.items(), key=lambda item: item[1], reverse=True))
    
# Salva nelle variabili i nomi e i valori delle prime e ultime 15 feature 
top_features = list(sorted_feature_importances.keys())[:15] + list(sorted_feature_importances.keys())[-15:]
top_coef_values = list(sorted_feature_importances.values())[:15] + list(sorted_feature_importances.values())[-15:]

# Colori delle barre
colors = ['darkblue' if val < 0 else 'skyblue' for val in top_coef_values]

# Plot delle coppie chiave-valore in ordine decrescente
plt.figure(figsize=(15, 5))
plt.bar(top_features, top_coef_values, color=colors, edgecolor='black')
plt.xlabel('Feature')
plt.ylabel('Importanza')
plt.xticks(rotation=90, fontsize=10)
plt.title('Significatività delle feature per la classificazione')
plt.grid(axis='x', alpha=0.4)
# Crea una lista di oggetti Patch per la legenda
legend_patches = [Patch(facecolor='darkblue', edgecolor='black', label='Significatività minore'), Patch(facecolor='skyblue', edgecolor='black', label='Significatività maggiore')]
# Crea la legenda personalizzata
plt.legend(handles=legend_patches)
plt.show()